In [2]:
import os
os.chdir('mmsegmentation')
os.getcwd()
import os
import numpy as np

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
template_path = 'LSR16F656/match/2.jpg'
img_path = 'LSR16F656/match/001554.tiff'
img_seg_path = 'LSR16F656/match/16f108.tiff'
img = cv2.imread(img_path) 
img_seg = cv2.imread(img_seg_path)
# img_seg = cv2.resize(img_seg, (890, 240))
template =cv2.imread(template_path, 0)
template = cv2.cvtColor(template, cv2.COLOR_GRAY2BGR)
# template = cv2.resize(template, (890, 240))

def find_nonzero_positions(arr):
    first_nonzero = None
    last_nonzero = None

    for i, value in enumerate(arr):
        if value != 0:
            if first_nonzero is None:
                first_nonzero = i
            last_nonzero = i

    return first_nonzero, last_nonzero

image1 = img_seg[:, :, 0]

image_row = np.sum(image1, axis=0)

image_col = np.sum(image1, axis=1)


# 找出第一个和最后一个非零元素的位置
first_row, last_row = find_nonzero_positions(image_row)
first_col, last_col = find_nonzero_positions(image_col)
width = last_row-first_row

image2 = template[:, :, 0]

image_row2 = np.sum(image2, axis=0)

image_col2 = np.sum(image2, axis=1)


# 找出第一个和最后一个非零元素的位置
first_row2, last_row2 = find_nonzero_positions(image_row2)
first_col2, last_col2 = find_nonzero_positions(image_col2)
width2 = last_row2-first_row2

# 打印结果
print("分割长：第一个非零元素的位置:", first_row)
print("分割长：最后一个非零元素的位置:", last_row)
print("分割长：", width)

print("模板长：第一个非零元素的位置:", first_row2)
print("模板长：最后一个非零元素的位置:", last_row2)
print("模板长：", width2)

gamma = width/width2

print(img_seg.shape)
print(template.shape)
print(gamma)

template = cv2.resize(template, (int(template.shape[1]*gamma), int(template.shape[0]*gamma)))
print(img_seg.shape)
print(template.shape)

# height, width = template.shape[:2]
# new_size = (width // 6, height // 6)
# template = cv2.resize(template, new_size, interpolation=cv2.INTER_LINEAR)

def cv_show(name,img):
    cv2.imshow(name,img)  # 传入自定义图像名，和图像变量
    cv2.waitKey(0)   # 图片不会自动消失
    # cv2.destroyWindow()   # 手动关闭窗口
# 显示图像
cv_show('img',img_seg)
cv_show('face',template)

: 

In [ ]:

methods_min = ['cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
           'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
# methods = ['cv2.TM_CCOEFF',]
for method in methods:
    method_index = eval(method)
    #（1）匹配单个对象
    # img代表原始图像，template代表模板窗口，1默认为cv2.TM_SQDIFF方法
    res = cv2.matchTemplate(img_seg, template, method_index) 
    # 获取结果的最值和最值位置
    min_val,max_val,min_loc,max_loc = cv2.minMaxLoc(res)
    top_left = min_loc if method in methods_min else max_loc
    # 最值位置是左上角的坐标位置，通过模板的宽和高可以在原图上把模板位置画出来
    h,w,_ = template.shape  # shape值是(高,宽)
    # 找出右下位置
    # top_left=(top_left[0]+15,top_left[1]+10)
    bottom_right = (top_left[0]+w,top_left[1]+h)
    # 复制一份图像，不然画框的时候原图像会变
    draw = img_seg.copy()
    # 输入图像画板draw，左上坐标，右下坐标
    cv2.rectangle(draw,top_left,bottom_right,(0,0,255),2)
    print(draw.shape)
    print(template.shape)
    crop = draw[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0], :] 
    dst=cv2.addWeighted(crop,0.75,template,0.25,0)
    draw[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0], :] = dst
    cv_show(f'{method}', draw)
    cv2.imwrite(f'D:\chaosheng\output\MMSegmentation_Tutorials-20230816\mmsegmentation\LSR16F656\match\{method}_match.png',draw)
    